# Developing a D-Wave Leap application, from problem selection to model implementation

- This notebook demonstrates an end to end process for users to develop D-Wave Leap application, from selecting a problem, through domain analysis to first model implementation.
- The process is simulated using a paper from the literature as a proxy for the problem owner.
    - This is done enable reproducibility while protecting privacy of users.
- Only the problem motivation and description from the paper is used.
- The formulation & implementation are developed independently to illustrate a practical approach to the process.
- Inline quotes from the paper should be read as a problem owner's responses to developer inquiries.
    - Quotes are sometimes reused when they answer multiple questions.

## Problem Selection

- The paper [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470) was selected.
- It was chosen based on criteria discussed below.
- An annotated CC-licensed [copy](./sustainability-13-03470-v2.pdf) is provided
- The paper builds on the prior [Kozłowski, E., Mazurkiewicz, D., Kowalska, B., Kowalski, D. (2018). Binary Linear Programming as a Decision-Making Aid for Water Intake Operators. In: Burduk, A., Mazurkiewicz, D. (eds) Intelligent Systems in Production Engineering and Maintenance – ISPEM 2017. ISPEM 2017. Advances in Intelligent Systems and Computing, vol 637. Springer, Cham.](https://doi.org/10.1007/978-3-319-64465-3_20)

### Is this a real problem?

> Supplying high-quality water at a competitive cost is a major challenge for water utilities worldwide, especially with ever-increasing water quality standards and energy prices [1,2]. Water supply systems are among the most important parts of infrastructure necessary to provide suitable quality of life for human beings.

> The issues of water production optimization and energy savings are part of the topic of sustainable development. 

### What is the ROI of solving this problem?

> There are many kinds of costs connected with water supplies. Among them, costs of electric power make an important share of operating costs because of using many electric-powered pumps necessary to bring water from its source or sources to customers

Based on the provided data, the power cost of operating all the pumps constanly is 288.30555 2015-USD per day, which is a very loose upper bound.
There is no data to quantify other benefits of optimization, such as reduced risk of reservoir constraint violations and reduced labor costs, if the pump scheduling were manual.

### Is there complete data for a self contained problem instance?
Complete data for pump capacities and power requirements as well as hourly water demand predictions and power costs are provided.

> The capacities and values of the electric power of the pumps are presented in Table 1

> The supplier of electric power does not use the same rate per MWh in its pricing policy all day long. Instead, it uses three tariff levels (see Table 2). 

> The example numerical data of the demand values for 24 timeslots of just one specific day are presented in Table 3 (along with corresponding electric power prices).

### What are the sources of complexity?
> Various prices of electric power make efficient controlling of the pumps much moredifficult because the requirements resulting from the demand levels as well as technical and safety conditions should be satisfied at the lowest cost.

### Who is the problem owner?

> The water supply system under consideration is that of a water supply operator based in a town with a population of about 25,000 inhabitants, located in Eastern Poland.

### What is the current solution to this problem?
> An example containing real-world input data was successfully solved using Microsoft Excel with a free OpenSolver add-in.

This would be a red flag in a real commercial application, as there is little benefit in a quantum implementation of a problem that is solved to optimality with a free Excel plugin.
This does affect the pedagogical value of this  problem and it is treated as a green field problem for the purposes of this exercise.

## Problem Description

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

> In order to optimize the operation of a water pumping station itself, it is necessary to create a pump schedule for some period of time (e.g., 24 h) that states when a given pump should be turned on and, if it is controllable, with what efficiency it should work (full capacity or below). Correct optimization requires defining many necessary constraints, including taking into account the predicted demand for water, which varies throughout the day, capacities of reservoir tanks and minimal volumes of water stored in them, costs of electrical power, etc.

### Detailed specification

> **The objective of the article is the minimization of the cost of electric power used by the pumps supplying water.**

> The water supply system under consideration is that of a water supply operator based in a town with a population of about 25,000 inhabitants, located in Eastern Poland. **The main parts of the system are wells, pumps, a reservoir tank, and the distribution pipeline network.**

> Supplied water is groundwater pumped from 7 wells. **The capacities and values of the electric power of the pumps are presented** in Table 1. **Water is pumped from the wells to a single reservoir tank with the capacity of Vmax** = 1500 m3 (the maximal volume of stored water).

> **The demand for water varies over time**. **The outflow of water from the reservoir tank via the distribution network to customers is a continuous process.** For practical reasons, **predictions of demand are made for 24 one-hour timeslots**.

> Controlling the pumps must obey the following requirements. **The pumps can operate with their nominal capacities only**, and the amount of water pumped by any pump depends on the time of operation only. **Each pump must operate for at least one hour per day.** Additionally, **at least one well and the pump integrated with it must be kept as a reserve at any moment of the day**. **The water inside the tank should be replaced at least once per day**. During standard operational conditions, **the volume of water in the reservoir tank cannot be less than Vmin** = 523.5 m3. It is the firefighting reserve, which is kept in order to satisfy an extra demand when a fire is extinguished by using water supplied from hydrants.

> ... **the supplier of electric power does not use the same rate per MWh in its pricing** policy all day long. Instead, it uses three tariff levels ...

> Various prices of electric power make efficient controlling of the pumps much moredifficult because the requirements resulting from the demand levels as well as technical and safety conditions should be satisfied at the lowest cost.

> As it has already been mentioned above, a basically continuous process of water distribution is approximately described in a discrete form, namely by specifying 24 predicted values of the demand for 24 one-hour times slots.

>  ... **in each timeslot, each pump can either be used or not**.

## Problem instance data

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

### Pump data (Table 1)

In [1]:
%%file pumps.csv
id capacity power_consumption
1 75 15
2 133 37
3 157 33
4 176 33
5 59 22
6 69 33
7 120 22

Overwriting pumps.csv


### Power tariffs (Table 2)

In [2]:
%%file tariff.csv
start end price
16 21 336.00
7 13 283.00
0 7 169.00
13 16 169.00
21 24 169.00

Overwriting tariff.csv


### Horly water demand and power prices on 1 August 2015 (Table 3)

In [3]:
%%file schedule.csv
time water_demand power_price
1 44.62 169
2 31.27 169
3 26.22 169
4 27.51 169
5 31.50 169
6 46.18 169
7 69.47 169
8 100.36 283
9 131.85 283
10 148.51 283
11 149.89 283
12 142.21 283
13 132.09 283
14 129.29 169
15 124.06 169
16 114.68 169
17 109.33 336
18 115.76 336
19 126.95 336
20 131.48 336
21 138.86 336
22 131.91 169
23 111.53 169
24 70.43 169

Overwriting schedule.csv


### Reservoir parameters

In [4]:
%%file reservoir.csv
Vmin Vmax Vinit
523.5 1500 550

Overwriting reservoir.csv


## Load data

In [5]:
import itertools as it

import numpy as np
import pandas as pd

import dimod
from dwave import system as dw

In [6]:
pumps = pd.read_csv("pumps.csv", sep=' ').set_index('id', drop=False)
schedule = pd.read_csv("schedule.csv", sep=' ').set_index('time', drop=False)
tariff = pd.read_csv("tariff.csv", sep=' ')
reservoir = pd.read_csv("reservoir.csv", sep=' ').loc[0]

## Define domain language

### Utilities

In [7]:
Sum = dimod.quicksum

### Inputs (Parameters)

#### *The capacities and values of the electric power of the pumps are presented ...*

In [8]:
def capacity(pump):
    return pumps.capacity[pump]

def power_consumption(pump):
    return pumps.power_consumption[pump]

#### *The supplier of electric power does not use the same rate per MWh in its pricing policy all day. It uses three tariff levels*

In [9]:
def power_price(time):
    return schedule.power_price[time]/1000

#### *The demand for water varies over time*

In [10]:
def water_demand(time):
    return schedule.water_demand[time]

### Outputs (Variables)

#### *In each timeslot, each pump can either be used or not*

In [11]:
def is_running(pump, time):
    return dimod.Binary(f"pump{pump}_time{time}")

#### The state of the system is is the reservoir volume (implicit)

In [12]:
def reservoir_volume(time): 
    return (
        dimod.Real(f"volume_time{time}")
        if time >= schedule.time.min()
        else reservoir.Vinit
    )

### Derived terms

#### *Water is pumped from the wells to a single reservoir tank*

In [13]:
def reservoir_inflow(time):
    return Sum(
            capacity(pump) * is_running(pump, time)
            for pump in pumps.id
        )

#### *The outflow of water from the reservoir tank via the distribution network to customers is a continuous process*

In [14]:
def reservoir_outflow(time):
    return (
        reservoir_volume(time-1) + reservoir_inflow(time)
        - reservoir_volume(time)
    )

#### Cost depends combined pump power use (implicit)

In [15]:
def power_used(time):
    return Sum(
            power_consumption(pump) * is_running(pump, time)
            for pump in pumps.id
        )

## Construct model

### Initialize model

In [16]:
model = dimod.CQM()

### Define objective

#### *The objective of the article is the minimization of the cost of electric power used by the pumps supplying water*

In [17]:
model.set_objective(
    Sum(
        power_price(time) * power_used(time)
        for time in schedule.time
    )
)

### Add constraints

#### *Each pump must operate for at least one hour per day*

In [18]:
for pump in pumps.id:
    model.add_constraint(
        Sum(
            is_running(pump, time) for time in schedule.time
        ) >= 1,
        f"pump{pump}_on_at_least_1h_per_day")

#### *At least one well and the pump integrated with it must be kept as a reserve at any moment of the day*

In [19]:
for time in schedule.time:
    model.add_constraint(
        Sum(
            is_running(pump, time) for pump in pumps.id
        ) <= pumps.shape[0] - 1,
        f"at_least_one_pump_in_reserve_at_time{time}" )

#### *The water inside the tank should be replaced at least once per day (?)*

This constraint illustrates a common the need for working with engaged problem owners. The meaning of the constraint is unclear. The only known means of removing water from the reservoir is user demand, which is not under the control of the operator. The solution in the paper also does not explicitly enforce this constraint. In a real scenario this would require further discussion with the problem owner.

#### *The outflow of water from the reservoir tank via the distribution network to customers is a continuous process*

In [20]:
for time in schedule.time:
    model.add_constraint(
        reservoir_outflow(time) == water_demand(time),
        f"volume_at_time{time}"
    )

#### *... a single reservoir tank with the capacity of Vmax*

In [21]:
for time in schedule.time:
    model.add_constraint(
        reservoir_volume(time) <= reservoir.Vmax,
        f"within_capacity_at_time{time}"
    )

#### *The volume of water in the reservoir tank cannot be less than Vmin*

In [22]:
for time in schedule.time:
    model.add_constraint(
        reservoir_volume(time) >= reservoir.Vmin,
        f"sufficient_reserve_at_time{time}"
    )

## Solve Model

In [23]:
sampler = dw.LeapHybridCQMSampler()

In [24]:
%%time
samples = sampler.sample_cqm(model, time_limit=20)
samples.resolve()

CPU times: user 890 ms, sys: 63.6 ms, total: 953 ms
Wall time: 21.1 s


In [25]:
feasible = samples.filter(lambda d: d.is_feasible)

In [26]:
feasible.to_pandas_dataframe(True)

,sample,energy,num_occurrences,is_satisfied,is_feasible
0,"{'pump1_time1': 1.0, 'pump1_time10': 1.0, 'pum...",99.419,1,"[True, True, True, True, True, True, True, Tru...",True
1,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",96.628,1,"[True, True, True, True, True, True, True, Tru...",True
2,"{'pump1_time1': 0.0, 'pump1_time10': 1.0, 'pum...",105.964,1,"[True, True, True, True, True, True, True, Tru...",True
3,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",104.998,1,"[True, True, True, True, True, True, True, Tru...",True
4,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",88.526,1,"[True, True, True, True, True, True, True, Tru...",True
5,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",102.412,1,"[True, True, True, True, True, True, True, Tru...",True
6,"{'pump1_time1': 1.0, 'pump1_time10': 0.0, 'pum...",101.307,1,"[True, True, True, True, True, True, True, Tru...",True
7,"{'pump1_time1': 0.0, 'pump1_time10': 0.0, 'pum...",155.129,1,"[True, True, True, True, True, True, True, Tru...",True


## Inspect model

In [27]:
lp_dump = dimod.lp.dumps(model)

In [28]:
print(lp_dump)

Minimize
 obj: + 2.535 pump1_time1 + 6.253 pump2_time1 + 5.577 pump3_time1 
 + 5.577 pump4_time1 + 3.7180000000000004 pump5_time1 + 5.577 pump6_time1 
 + 3.7180000000000004 pump7_time1 + 2.535 pump1_time2 + 6.253 pump2_time2 
 + 5.577 pump3_time2 + 5.577 pump4_time2 + 3.7180000000000004 pump5_time2 
 + 5.577 pump6_time2 + 3.7180000000000004 pump7_time2 + 2.535 pump1_time3 
 + 6.253 pump2_time3 + 5.577 pump3_time3 + 5.577 pump4_time3 
 + 3.7180000000000004 pump5_time3 + 5.577 pump6_time3 
 + 3.7180000000000004 pump7_time3 + 2.535 pump1_time4 + 6.253 pump2_time4 
 + 5.577 pump3_time4 + 5.577 pump4_time4 + 3.7180000000000004 pump5_time4 
 + 5.577 pump6_time4 + 3.7180000000000004 pump7_time4 + 2.535 pump1_time5 
 + 6.253 pump2_time5 + 5.577 pump3_time5 + 5.577 pump4_time5 
 + 3.7180000000000004 pump5_time5 + 5.577 pump6_time5 
 + 3.7180000000000004 pump7_time5 + 2.535 pump1_time6 + 6.253 pump2_time6 
 + 5.577 pump3_time6 + 5.577 pump4_time6 + 3.7180000000000004 pump5_time6 
 + 5.577 pump6_

In [29]:
with open("reservoir.lp", "w") as f:
    print(lp_dump, file=f)